In [1]:
import os
import sys
import clr
import sys
import numpy as np
import cv2
import time
from thorlabs_tsi_sdk.tl_camera import TLCameraSDK, OPERATION_MODE
import ctypes
import pickle

In [2]:
clr.AddReference("C:\\Program Files\\Thorlabs\\Kinesis\\Thorlabs.MotionControl.DeviceManagerCLI.dll")
clr.AddReference("C:\\Program Files\\Thorlabs\\Kinesis\\Thorlabs.MotionControl.GenericMotorCLI.dll")
clr.AddReference("C:\\Program Files\\Thorlabs\\Kinesis\\ThorLabs.MotionControl.KCube.InertialMotorCLI.dll")
from Thorlabs.MotionControl.DeviceManagerCLI import *
from Thorlabs.MotionControl.GenericMotorCLI import *
from Thorlabs.MotionControl.KCube.InertialMotorCLI import *
from System import Decimal
dll_path = r'C:\Users\athar\OneDrive\Desktop\Unrelated\Integrated\thorlabs_tsi_camera_sdk.dll'
my_dll = ctypes.WinDLL(dll_path)

In [3]:
def main():
    """The main entry point for the application"""

    # Uncomment this line if you are using
    SimulationManager.Instance.InitializeSimulations()

    try:

        DeviceManagerCLI.BuildDeviceList()
        image_list = []
        # create new device
        serial_no = "74000948"  # Replace this line with your device's serial number

        device = KCubeInertialMotor.CreateKCubeInertialMotor(serial_no)

        # Connect
        device.Connect(serial_no)
        time.sleep(0.25)


        # Ensure that the device settings have been initialized
        if not device.IsSettingsInitialized():
            device.WaitForSettingsInitialized(10000)  # 10 second timeout
            assert device.IsSettingsInitialized() is True

        # Get Device Information and display description
        device_info = device.GetDeviceInfo()
        print(device_info.Description)
        # Start polling and enable channel
        device.StartPolling(250)  #250ms polling rate
        time.sleep(25)
        device.EnableDevice()
        time.sleep(0.25)  # Wait for device to enable

        # Load any configuration settings needed by the controller/stage
        inertial_motor_config = device.GetInertialMotorConfiguration(serial_no)

        # Get parameters related to homing/zeroing/moving
        device_settings = ThorlabsInertialMotorSettings.GetSettings(inertial_motor_config)

        # Step parameters for an intertial motor channel
        chan1 = InertialMotorStatus.MotorChannels.Channel1  # enum chan ident
        device_settings.Drive.Channel(chan1).StepRate = 500
        device_settings.Drive.Channel(chan1).StepAcceleration = 100000

        # Send settings to the device
        device.SetSettings(device_settings, True, True)

        # Home or Zero the device (if a motor/piezo)
        print("Zeroing device")
        device.SetPositionAs(chan1, 0)

        # Move the device to a new position
        '''
        There are two versions of each movement Method in the API,
        with the same names but different inputs. 
        
        Methods that take an integer argument as an input move in terms of 
        device steps (step size can be user-defined). These methods are 
        used in open-loop operation.
        
        Methods that take a Decimal argument as an input move in real units.
        These methods are used in open closed-loop operation.        
        '''
        if not os.path.exists('images'):
            os.makedirs('images')
        for i in range(1,20):
            NUM_FRAMES = 10  # adjust to the desired number of frames
            try:
                # if on Windows, use the provided setup script to add the DLLs folder to the PATH
                from windows_setup import configure_path
                configure_path()
            except ImportError:
                configure_path = None

            with TLCameraSDK() as sdk:
                available_cameras = sdk.discover_available_cameras()
                if len(available_cameras) < 1:
                    print("no cameras detected")
                with sdk.open_camera(available_cameras[0]) as camera:
                    camera.exposure_time_us = 10000  # set exposure to 11 ms
                    camera.frames_per_trigger_zero_for_unlimited = 0  # start camera in continuous mode
                    camera.image_poll_timeout_ms = 1000  # 1 second polling timeout
                    camera.arm(2)
                    camera.issue_software_trigger()
                    frame = camera.get_pending_frame_or_null()
                    if frame is not None:
                        print("frame #{} received!".format(frame.frame_count))
                        frame.image_buffer
                        image_buffer_copy = np.copy(frame.image_buffer)
                        numpy_shaped_image = image_buffer_copy.reshape(camera.image_height_pixels, camera.image_width_pixels)
                        nd_image_array = np.full((camera.image_height_pixels, camera.image_width_pixels, 3), 0, dtype=np.uint8)
                        nd_image_array[:,:,0] = numpy_shaped_image
                        nd_image_array[:,:,1] = numpy_shaped_image
                        nd_image_array[:,:,2] = numpy_shaped_image
                        
                        cv2.imshow("Image From TSI Cam", nd_image_array)
                        image_filename = f'images/img_{i*10}.png'
                        cv2.imwrite(image_filename, nd_image_array)
                    else:
                        print("Unable to acquire image, program exiting...")
                        exit()
                        
                    cv2.waitKey(1)
                    print("1st loop")
                    camera.disarm()
            # new_pos = Decimal(10.0)
            # uncomment the following for open-loop operation:
            new_pos = int(10)*i
            print(f'Moving to position {new_pos}')
            # Pythonnet will infer which method to use:
            device.MoveTo(chan1, new_pos, 60000)  # 60 second timeout
            print("Move Complete")
            # new_pos = Decimal(20.0)
            time.sleep(1)
        # filename = 'image_data.pkl'
        # with open(filename, 'wb') as file:
        #     pickle.dump(image_list, file)
        #     print(f"Image data saved to {filename}")
        # Stop Polling and Disconnect
        device.StopPolling()
        device.Disconnect()

    except Exception as e:
        print(e)

    # Uncomment this line if you are using Simulations
    #SimulationManager.Instance.UninitializeSimulations()
    ...


#  Because we are using the 'with' statement context-manager, disposal has been taken care of.
print("program completed")

program completed


In [4]:
if __name__ == "__main__":
    main()

Piezo Motor Controller
Zeroing device
frame #1 received!
1st loop
Moving to position 10
Move Complete
frame #1 received!
1st loop
Moving to position 20
Move Complete
frame #1 received!
1st loop
Moving to position 30
Move Complete


: 